In [6]:
from typing import Any, Dict, List, Optional,Set, Tuple 
import numpy as np
import pandas as pd
from random import sample,random
from math import floor
import matplotlib.pyplot as plt
!gdown --id 1xB6ewaR3HMKzhMJgIgvMWHNODifHDHUJ
!gdown --id 1OP0-auufRe8cS_S7KVtiG15XIUglILze
def generate_data(size : int = 100,coefs :int = 4) -> np.array:
    """
    Untuk Dummy saja
    Generate Random data < Continous >
    size = How many observations
    coefs = How many features / how many chromosome
    Returns:
        np array (x), (y)
    """
    coeffs : List[float] = [0.4, 0.3 ,0.2, -0.1]
    x : List[np.array] = [[np.random.normal() for _ in range(len(coeffs))] for _ in range(size) ]
    y : List[np.array] = [np.dot(i,coeffs) for i in x]
    
    return np.array(x), np.array(y)



class Population:
    
    def __init__(self,TotalSize,Totalgenome: int) -> None:
        self.Totalgenome: int = Totalgenome
        self.totalSize : int = TotalSize
        self.bestIndividuals : List =[]
        self.population = []
        self.fitnessed  : Dict[str,Any]

    def createIndividu(self) -> np.array:
        """
        Create Individu using Real Numbers

        Returns:
            List[int]: Real numbers sequence
        """
        return np.random.normal(size=self.Totalgenome)
    
    def createPopulation(self) -> None :
        """
        Create Population with x size Depends on total size
        and append it to population list 
        
        Returns:
            None 
        """
        self.population.append([self.createIndividu() for _ in range(self.totalSize)])
   
    def fitness(self, individual , inputs : np.array, yTrue : np.array) -> Dict[str,Any]:
        """
        
        Fitness current Individual

        Args:
            individual (List[int]) --> Chromosome on a population
            inputs (np.array) ->> Input data
            yTrue (np.array) --> Groud truth
        Returns:
            Dict[str,Any]: [description]
        """
        info : Dict[str,Any] = {}
        bias : np.float = 0
        predicted : np.array =  bias + np.dot(inputs,individual) 
        yTrue_mean : np.float = np.mean(yTrue) 

        SST : np.float = np.sum(np.array([(y - yTrue_mean) ** 2 for y in yTrue]),axis=None)
        SSR : np.float = np.sum(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)]),axis = None)
    
        bias :np.float = np.mean(np.sqrt(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)])))
        RMSE : np.float = np.sqrt(SSR/ len(y))
        Rsquared : np.float = (1 - (SSR / SST))

        SSE : np.float = SSR / len(y)
        oneDivError : np.float = 1 / SSE

        info["Rsquared"] = Rsquared
        info["coeff"] = individual
        info["error"] = SSE
        info["RMSE"] = RMSE 
        info["divByOne"] = oneDivError
        info["bias"] = bias 
        self.fitnessed = info
        return info
    
    def evaluate_population(self, x:np.array ,y:np.array ,selectionSize : int) -> None :
        """
        Function to evaluate the best individual from current population

        Args:
           x (np.array) , Input data
           y (np.array, Ground truth
           selectionSize (int) : How many best Individuals you want to grab from the population

        Returns:
            None
        """
        fitness_list= [self.fitness(individual,x,y) for individual in self.population[0]]
        error_list : List[Dict[str,Any]] = sorted(fitness_list,key=lambda i : i["error"])
        best_individuals = error_list[: selectionSize]
        self.bestIndividuals.clear()
        self.bestIndividuals.append(best_individuals)
        
        print(f"Error {best_individuals[0]['error']}\n RSquared {best_individuals[0]['Rsquared']}\n") 


    def mutate(self,individual : List[float], probabilityMutating : float) -> List[float]:
        """
        
        Ini Fungsinya lakuin mutasi 

        Args:
            individual (List[float]): Individu pada populasi
            probabilityMutating (float): Probabilitas Individu saat ini bermutasi 

        Returns:
            List[float]: Individu Yang telah bermutasi < Jika dilakukan >
        """
        indx : List[int] = [i for i in range(len(individual))]

        totalMutatedGens : int = int(probabilityMutating * len(individual))
        indx_toMutate : List[int] = sample(indx,k = totalMutatedGens)
        for ind in indx_toMutate:
            choice : np.int = np.random.choice([-1,1])
            gene_transform : float = choice*random()

            individual[ind] = individual[ind] + gene_transform
        return individual
        
    def crossover(self, parent1 : Dict[str,Any], parent2 : Dict[str,Any]) -> None:
        """
           CrossOver / persilangan untuk menghasilkan keturunan 
           metode yang digunakan adalah Simple Arithmetic crossover
        Args:
            parent1 (Dict[str,Any]): [description]
            parent2 (Dict[str,Any]): [description]
        Return: None
        """
        
        anak_keturunan : Dict[int,Any] = {}
        index : List[int] = [i for i in range( self.Totalgenome )]
        indexRandomize : List[int] = sample(index, floor(0.5 * self.Totalgenome))
        IndexNotInRandomize : List[int] = [i for i in index if i not in indexRandomize]

        getCromosomeFromParent1 : List[Any] = [[i,parent1['coeff'][i]] for i in indexRandomize]
        getCromosomeFromParent2 : List[Any] = [[i,parent2["coeff"][i]] for i in IndexNotInRandomize]
        
        anak_keturunan.update({key :value for (key,value) in getCromosomeFromParent1})
        anak_keturunan.update({key : value for(key,value) in getCromosomeFromParent2})
    
        return [anak_keturunan[i] for i in index]

    def create_new_generation(self,probabilityMutating : float,probability_indiv_mutating:float) -> None:
        """
        Create new population using the best individuals

        Args:
            probabilityMutating(float) : 
            probability_indiv_mutating (float) : 

        Return:
            None
        """
        pasangan_sah = [sample(self.bestIndividuals[0],2) for _ in range( self.totalSize)]
        crossOverered_parents = [self.crossover(pasangan[0],pasangan[1]) for pasangan in pasangan_sah]
        pasangan_sah_indx = [i for i in range(self.totalSize)]
        pasanganCalonMutasi = sample(pasangan_sah_indx,floor(probabilityMutating * self.totalSize))

        PasanganMutasi = [[i,self.mutate(crossOverered_parents[i],probability_indiv_mutating)] for i in pasanganCalonMutasi]
        for anakMutasi in PasanganMutasi:
            crossOverered_parents[anakMutasi[0]] = anakMutasi[1]
        self.population.clear()
        self.population.append(crossOverered_parents)
    
if __name__ == "__main__":
    x,y = generate_data(100)
    np.random.seed(123) 
    pop =Population(100,4)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.8
    probability_indiv_mutation = 0.25

    for i in range(3) :
        print(f"generation : {i}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        
    
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])
    

    #parent1 = pop.fitness(individual,x,y)
    #parent2 = pop.fitness(individual,x,y)

Downloading...
From: https://drive.google.com/uc?id=1xB6ewaR3HMKzhMJgIgvMWHNODifHDHUJ
To: /content/DatasetRequired.csv
100% 8.82k/8.82k [00:00<00:00, 22.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OP0-auufRe8cS_S7KVtiG15XIUglILze
To: /content/saham.csv
100% 95.1k/95.1k [00:00<00:00, 35.5MB/s]
generation : 0
Error 0.6473488175203902
 RSquared -0.9290148742979771

generation : 1
Error 0.02313605454860628
 RSquared 0.931057581092399

generation : 2
Error 0.0197092318792916
 RSquared 0.9412690647960535

Nilai x :  [ 0.3115815  -0.14205073  0.75203367  0.06964075]
Nilai Y :  0.2254600419193178
Predicted =  0.30850860521085166
{'Rsquared': 0.9412690647960535, 'coeff': [0.4115421526449954, 0.3696195863041741, 0.1771923291989879, -0.21794626210294177], 'error': 0.0197092318792916, 'RMSE': 0.14038957183242493, 'divByOne': 50.7376444766828, 'bias': 0.11470775730403517}


In [7]:
def terminate(ans : Dict[str,Any]):
    """
    Terminate Genetic Algorithm Menggunakan nilai RSquared yang telah didefinisiskan
    
    ans : Dict - > Berisi nilai Rsquared,coeff,RMSE ...
    
    return 
    True jika Rsquared > 0.95
    False others
    """
    if ans["Rsquared"] > 0.95:
        return True
    else :
        return False

In [8]:
from sklearn.preprocessing import StandardScaler
StdScaler = StandardScaler()


In [9]:
"""
Codingan Ini digunakan untuk preprocessing data saham dari link berikut : https://github.com/wildangunawan/Dataset-Saham-IDX
menjadi nilai open selama 10 hari berturut turut
"""
df = pd.read_csv("saham.csv")
dfnew = pd.DataFrame()
yow = []
def generate_newdataset():
    k=13
    j=1
    for i in range(100):
        
        dict1 = {}
        dict1.update( {i:df[j:k]["open_price"].values})
        j+=1
        k+=1
        yow.append(dict1)
generate_newdataset()
dfnew = pd.DataFrame(yow[0])
for i in range(1,len(yow)):
    dfnew[i] = pd.DataFrame(yow[i])
dfnew = dfnew.T
dfnew.to_csv("DatasetRequired.csv")


In [10]:
if __name__ == "__main__":
    df = pd.read_csv("DatasetRequired.csv")
    df = df.iloc[:,1:13]
    x = df.iloc[:,1:12]
    y = df.iloc[:,11]
    
    y = StdScaler.fit_transform(np.array(y).reshape(-1,1))
    x = StdScaler.fit_transform(x) 
    np.random.seed(123) 
    pop =Population(100,11)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.2
    probability_indiv_mutation = 0.25

    terminate_ga = False
    generation = 0
    while(terminate_ga != True): 
        print(f"generation : {generation}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        terminate_ga = terminate(pop.fitnessed)
        generation += 1
        
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])

generation : 0
Error 1.0696958324097
 RSquared -0.06969583240969945

generation : 1
Error 0.3873795195401703
 RSquared 0.6126204804598299

generation : 2
Error 0.26782499148546907
 RSquared 0.732175008514531

generation : 3
Error 0.16513546216394082
 RSquared 0.8348645378360592

generation : 4
Error 0.0799060192764779
 RSquared 0.9200939807235221

generation : 5
Error 0.16513546216394082
 RSquared 0.8348645378360592

generation : 6
Error 0.13618248238590935
 RSquared 0.8638175176140906

generation : 7
Error 0.13400644565715822
 RSquared 0.8659935543428419

generation : 8
Error 0.12988900103832157
 RSquared 0.8701109989616784

generation : 9
Error 0.12760440181746538
 RSquared 0.8723955981825346

generation : 10
Error 0.12760440181746538
 RSquared 0.8723955981825346

generation : 11
Error 0.1262795209837211
 RSquared 0.8737204790162789

generation : 12
Error 0.12089348078511453
 RSquared 0.8791065192148855

generation : 13
Error 0.11205085505906778
 RSquared 0.8879491449409322

generati

In [11]:

predicted = pop.bestIndividuals[0][0]['bias'] + np.dot(x, pop.bestIndividuals[0][0]["coeff"])
predicted

array([ 1.89995384,  1.34096476,  1.23540845,  0.78370746,  1.08133611,
        1.04383809,  0.84387567,  0.78324597,  0.5249303 ,  0.06218159,
       -0.35714195,  0.08450826,  0.29260711,  0.54717498,  0.48920042,
        0.16799996,  0.07542503, -0.12573574,  0.12468877, -0.5181359 ,
       -0.26822244,  0.57007581,  1.01212131,  0.36826759, -0.56389077,
        0.23874818,  0.53753813,  0.3306637 , -0.42749464, -0.26747885,
        0.4302296 ,  0.01833062, -0.30975064, -0.69597848, -0.25783104,
       -0.17485259, -1.24497026, -1.72163844, -2.12856891, -1.9152222 ,
       -1.90044503, -2.0333948 , -1.56072413, -1.10817285, -1.04527847,
       -1.49202702, -1.31144869, -0.83101555, -0.94524051, -0.49088293,
       -0.59844397,  0.20397003,  0.93138103,  0.22932917,  0.2434445 ,
        0.07128079,  0.39600332, -0.04625902, -0.31047864, -0.41610785,
        0.63079622,  0.41391715, -0.37147434,  0.12668701,  0.08676144,
        0.08843665, -0.59082154, -0.333702  , -0.43004026,  0.34

In [12]:
%whos

Variable                     Type                          Data/Info
--------------------------------------------------------------------
Any                          _SpecialForm                  typing.Any
Dict                         _GenericAlias                 typing.Dict
List                         _GenericAlias                 typing.List
Optional                     _SpecialForm                  typing.Optional
Population                   type                          <class '__main__.Population'>
Set                          _GenericAlias                 typing.Set
StandardScaler               type                          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
StdScaler                    StandardScaler                StandardScaler(copy=True,<...>mean=True, with_std=True)
Tuple                        _VariadicGenericAlias         typing.Tuple
df                           DataFrame                              0       1       <...>\n[100 rows x 12 columns]